# Exploring the relationship between gender and policing

## Do the genders commit different violations?

In [6]:
import pandas as pd
ri = pd.read_csv("police.csv")
ri.drop(["county_name", "state"], axis="columns", inplace=True)
ri.dropna(subset=["driver_gender"], inplace=True)
ri["is_arrested"] = ri.is_arrested.astype("bool")
combined = ri.stop_date.str.cat(ri.stop_time, sep=" ")
ri["stop_datetime"] = pd.to_datetime(combined)
ri.set_index("stop_datetime", inplace=True)

print(ri.violation.value_counts())
print(ri.violation.value_counts(normalize=True))

Speeding               48423
Moving violation       16224
Equipment              10921
Other                   4409
Registration/plates     3703
Seat belt               2856
Name: violation, dtype: int64
Speeding               0.559571
Moving violation       0.187483
Equipment              0.126202
Other                  0.050950
Registration/plates    0.042791
Seat belt              0.033004
Name: violation, dtype: float64


### Comparing violations by gender

In [8]:
female = ri[ri["driver_gender"] == "F"]
male = ri[ri["driver_gender"] == "M"]
print(female.violation.value_counts(normalize=True))
print(male.violation.value_counts(normalize=True))

Speeding               0.658114
Moving violation       0.138218
Equipment              0.105199
Registration/plates    0.044418
Other                  0.029738
Seat belt              0.024312
Name: violation, dtype: float64
Speeding               0.522243
Moving violation       0.206144
Equipment              0.134158
Other                  0.058985
Registration/plates    0.042175
Seat belt              0.036296
Name: violation, dtype: float64


## Does gender affect who gets a ticket for speeding?

### Comparing speeding outcomes by gender

In [32]:
female_and_speeding = ri[(ri["driver_gender"] == "F") & (ri["violation"] == "Speeding")]
male_and_speeding = ri[(ri["driver_gender"] == "M") & (ri["violation"] == "Speeding")]
print(female_and_speeding["stop_outcome"].value_counts(normalize=True))
print(male_and_speeding["stop_outcome"].value_counts(normalize=True))

Citation            0.952192
Warning             0.040074
Arrest Driver       0.005752
N/D                 0.000959
Arrest Passenger    0.000639
No Action           0.000383
Name: stop_outcome, dtype: float64
Citation            0.944595
Warning             0.036184
Arrest Driver       0.015895
Arrest Passenger    0.001281
No Action           0.001068
N/D                 0.000976
Name: stop_outcome, dtype: float64


The numbers are similar for males and females. Data fails to show that gender has an impact on who gets a ticket for speeding.

## Does gender affect whose vehicle is searched?

In [35]:
print(ri.search_conducted.dtype)
print(ri.search_conducted.value_counts(normalize=True))
print(ri.search_conducted.mean())

bool
False    0.961785
True     0.038215
Name: search_conducted, dtype: float64
0.0382153092354627


### Comparing search rates by gender

In [38]:
print(ri[ri.driver_gender == "F"].search_conducted.mean())
print(ri[ri.driver_gender == "M"].search_conducted.mean())
print(ri.groupby("driver_gender").search_conducted.mean())

0.019180617481282074
0.04542557598546892
driver_gender
F    0.019181
M    0.045426
Name: search_conducted, dtype: float64


### Adding a second factor to the analysis

In [62]:
print(ri.groupby(["driver_gender", "violation"]).search_conducted.mean())
print(ri.groupby(["violation", "driver_gender"]).search_conducted.mean())

driver_gender  violation          
F              Equipment              0.039984
               Moving violation       0.039257
               Other                  0.041018
               Registration/plates    0.054924
               Seat belt              0.017301
               Speeding               0.008309
M              Equipment              0.071496
               Moving violation       0.061524
               Other                  0.046191
               Registration/plates    0.108802
               Seat belt              0.035119
               Speeding               0.027885
Name: search_conducted, dtype: float64
violation            driver_gender
Equipment            F                0.039984
                     M                0.071496
Moving violation     F                0.039257
                     M                0.061524
Other                F                0.041018
                     M                0.046191
Registration/plates  F                0.05492

For all types of violations, the search rate is higher for males than for females.

## Does gender affect who is frisked during a search?

### Counting protective frisks

In [66]:
print(ri.search_type.value_counts())
ri["frisk"] = ri.search_type.str.contains("Protective Frisk", na=False)
print(ri.frisk.dtype)
print(ri.frisk.sum())

Incident to Arrest                                          1290
Probable Cause                                               924
Inventory                                                    219
Reasonable Suspicion                                         214
Protective Frisk                                             164
Incident to Arrest,Inventory                                 123
Incident to Arrest,Probable Cause                            100
Probable Cause,Reasonable Suspicion                           54
Probable Cause,Protective Frisk                               35
Incident to Arrest,Inventory,Probable Cause                   35
Incident to Arrest,Protective Frisk                           33
Inventory,Probable Cause                                      25
Protective Frisk,Reasonable Suspicion                         19
Incident to Arrest,Inventory,Protective Frisk                 18
Incident to Arrest,Probable Cause,Protective Frisk            13
Inventory,Protective Fris

### Comparing frisk rates by gender

In [68]:
searched = ri[ri.search_conducted == True]
print(searched.frisk.mean())
print(searched.groupby("driver_gender")["frisk"].mean())

0.09162382824312065
driver_gender
F    0.074561
M    0.094353
Name: frisk, dtype: float64


The frisk rate is higher for males than for females.